In [1]:
import json

In [2]:
with open("dev-v1.1.json", "r", encoding="utf-8") as file:
    squad_data = json.load(file)

In [3]:
squad_data.keys()

dict_keys(['data', 'version'])

In [4]:
def print_json_structure(data, indent=0):
    prefix = " " * indent
    if isinstance(data, dict):
        for key, value in data.items():
            print(f"{prefix}- {key}: {type(value).__name__}")
            print_json_structure(value, indent + 2)
    elif isinstance(data, list):
        print_json_structure(data[0], indent + 2)

In [5]:
print_json_structure(squad_data)

- data: list
    - title: str
    - paragraphs: list
        - context: str
        - qas: list
            - answers: list
                - answer_start: int
                - text: str
            - question: str
            - id: str
- version: str


In [6]:
import random

In [7]:
input_data = []
output_data = []

In [8]:
for entry in squad_data['data']:
    for paragraph in entry['paragraphs']:
        context = paragraph['context']
        for qa in paragraph['qas']:
            question = qa['question']
            answer_text = random.choice(qa['answers'])['text']
            
            input_data.append(context + " " + question)
            output_data.append(answer_text)

In [9]:
input_data[:5]

['Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50. Which NFL team represented the AFC at Super Bowl 50?',
 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (A

In [10]:
output_data[:5]

['Denver Broncos',
 'Carolina Panthers',
 "Levi's Stadium",
 'Denver Broncos',
 'gold']

In [11]:
import gensim.downloader as api

In [12]:
word2vec = api.load("word2vec-google-news-300")

In [13]:
word2vec.key_to_index["And"]

169

In [15]:
import tensorflow as tf

In [16]:
import numpy as np

In [17]:
encoder_inputs = input_data

start_token = "<start>"
end_token = "<end>"

decoder_inputs = [f"{start_token} {seq} {end_token}" for seq in output_data]
decoder_outputs = [f"{seq} {end_token}" for seq in output_data]

encoder_inputs[0], decoder_inputs[0], decoder_outputs[0]

('Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50. Which NFL team represented the AFC at Super Bowl 50?',
 '<start> Denver Broncos <end>',
 'Denver Broncos <end>')

In [18]:
max_len = max(max([len(seq.split()) for seq in input_data]), max([len(seq.split()) for seq in output_data]))

In [21]:
text_vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=15000,
    output_mode="int",
    standardize=lambda x: tf.strings.regex_replace(x, '[^ a-zA-Z0-9<>\']', ''),
    output_sequence_length=max_len,
    split="whitespace",
    pad_to_max_tokens=True
)

In [22]:
text_vectorizer.adapt(encoder_inputs + decoder_inputs)

In [23]:
vocabulary = text_vectorizer.get_vocabulary()

In [24]:
len(vocabulary), vocabulary[:5], vocabulary[14995:], vocabulary.index("<start>"), vocabulary.index("<end>")

(15000,
 ['', '[UNK]', 'the', 'of', 'and'],
 ['suspect', 'supplemented', 'successes', 'studentteacher', 'stature'],
 14,
 15)

In [25]:
text_vectorizer(encoder_inputs[0]).shape, text_vectorizer(encoder_inputs[0])[:140]

(TensorShape([651]),
 <tf.Tensor: shape=(140,), dtype=int64, numpy=
 array([  56,   49,  229,    9,   25,  180, 1798,  162,    6, 1370,    2,
        1285,    3,    2,  292, 1002, 1272,  370,   13,    2,  713,  183,
          10,  180, 1002,  615, 1211, 1285,  480,  284,  899,    2,  292,
        1002,  615, 1079, 1285,  453,  277, 3834,    6, 1922,   33,  304,
          56,   49,  809,   10,  162,    9,  640,   18,  574,  533, 1133,
          24, 1405,  882,    5,    2,  197,  770,  874, 1308,   24, 1262,
        2239,  256,  176,   38,    9,    2, 1635,   56,   49,    2,  866,
        3867,    2, 4641, 1624,   17,  235, 4366, 2894,   11,  111,   11,
        2427, 5451,    2, 1411,    3, 3859,  132,   56,   49,  162,   17,
         392, 1347,  125,   21,    2,  162,   59,   31,   51,   80,   11,
          56,   49, 1591,  178,   12,    2,  681,  121, 4940,  986,    2,
        2919, 1347,  229,  323,  370,  225,  735,    2, 1211,   24,   56,
          49,  229,    0,    0,    0,    0, 

In [26]:
vocab_size = len(text_vectorizer.get_vocabulary())

In [27]:
embedding_dim = 300
unk_vector = np.random.uniform(-0.05, 0.05, 300)
start_vector = np.random.uniform(-0.05, 0.05, embedding_dim)
end_vector = np.random.uniform(-0.05, 0.05, embedding_dim)
embedding_matrix = np.zeros((vocab_size, embedding_dim))
not_found_words = []

vocab = text_vectorizer.get_vocabulary()
word_to_index = {word: idx for idx, word in enumerate(vocab)}

for word, idx in word_to_index.items():
    if word == '':
        continue
    elif word in word2vec:
        embedding_matrix[idx] = word2vec[word]
    elif word.capitalize() in word2vec:
        embedding_matrix[idx] = word2vec[word.capitalize()]
    elif word.lower() in word2vec:
        embedding_matrix[idx] = word2vec[word.lower()]
    elif word == "<start>":
        not_found_words.append(idx)
        embedding_matrix[idx] = start_vector
    elif word == "<end>":
        not_found_words.append(idx)
        embedding_matrix[idx] = end_vector
    else:
        not_found_words.append(idx)
        embedding_matrix[idx] = unk_vector

In [28]:
len(not_found_words), not_found_words[:10]

(1735, [1, 14, 15, 229, 272, 275, 385, 464, 477, 487])

In [29]:
embedding_matrix[0][:3], embedding_matrix[229][:3], embedding_matrix[272][:3], embedding_matrix[14][:3], embedding_matrix[15][:3]

(array([0., 0., 0.]),
 array([ 0.01292978, -0.0196126 ,  0.03690822]),
 array([ 0.01292978, -0.0196126 ,  0.03690822]),
 array([ 0.04439599, -0.03755883, -0.01799788]),
 array([ 0.04773822, -0.03918727, -0.02714973]))

In [30]:
embedding_layer = tf.keras.layers.Embedding(
    input_dim=vocab_size,
    output_dim=embedding_dim,
    weights=[embedding_matrix],
    input_length=max_len,
    mask_zero=True,
    trainable=False
)

In [31]:
encoder_inputs_vec = text_vectorizer(encoder_inputs)
decoder_inputs_vec = text_vectorizer(decoder_inputs)
decoder_outputs_vec = text_vectorizer(decoder_outputs)

In [32]:
from sklearn.model_selection import train_test_split

In [33]:
encoder_inputs_vec

<tf.Tensor: shape=(10570, 651), dtype=int64, numpy=
array([[ 56,  49, 229, ...,   0,   0,   0],
       [ 56,  49, 229, ...,   0,   0,   0],
       [ 56,  49, 229, ...,   0,   0,   0],
       ...,
       [ 10,   1,  35, ...,   0,   0,   0],
       [ 10,   1,  35, ...,   0,   0,   0],
       [ 10,   1,  35, ...,   0,   0,   0]])>

In [34]:
encoder_inputs_vec = encoder_inputs_vec.numpy()
decoder_inputs_vec = decoder_inputs_vec.numpy()
decoder_outputs_vec = decoder_outputs_vec.numpy()

encoder_input_train, encoder_input_val, decoder_input_train, decoder_input_val, decoder_output_train, decoder_output_val = train_test_split(
    encoder_inputs_vec, decoder_inputs_vec, decoder_outputs_vec, test_size=0.2, random_state=42)

encoder_input_val, encoder_input_test, decoder_input_val, decoder_input_test, decoder_output_val, decoder_output_test = train_test_split(
    encoder_input_val, decoder_input_val, decoder_output_val, test_size=0.5, random_state=42)

encoder_input_train.shape, encoder_input_val.shape, encoder_input_test.shape

((8456, 651), (1057, 651), (1057, 651))

In [35]:
encoder_input_train[0][:6], decoder_input_train[0][:6], decoder_output_train[0][:6]

(array([  10, 5358,    5,    2, 2453,  611]),
 array([  14,  304, 4084,  784,   15,    0]),
 array([ 304, 4084,  784,   15,    0,    0]))

In [38]:
def masked_accuracy(y_true, y_pred):
    mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
    correct = tf.cast(tf.equal(y_true, tf.argmax(y_pred, axis=-1)), tf.float32)
    return tf.reduce_sum(correct * mask) / tf.reduce_sum(mask)

In [40]:
# Encoder
encoder_input = tf.keras.layers.Input(shape=(max_len,))
encoder_embedding = embedding_layer(encoder_input)
encoder_masking = tf.keras.layers.Masking(mask_value=0)(encoder_embedding)
_, forward_h, forward_c = tf.keras.layers.LSTM(512, return_state=True)(encoder_masking)
encoder_states = [forward_h, forward_c]

# Decoder
decoder_input = tf.keras.layers.Input(shape=(max_len,))
decoder_embedding = embedding_layer(decoder_input)
decoder_masking = tf.keras.layers.Masking(mask_value=0)(decoder_embedding)
decoder_vectors, _, _ = tf.keras.layers.LSTM(512, return_sequences=True, return_state=True)(decoder_masking, initial_state=encoder_states)
decoder_output = tf.keras.layers.Dense(vocab_size, activation="softmax")(decoder_vectors)

model = tf.keras.Model([encoder_input, decoder_input], decoder_output)

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=[masked_accuracy])

model.summary()

model.fit([encoder_input_train, decoder_input_train], decoder_output_train, validation_data=([encoder_input_val, decoder_input_val], decoder_output_val), epochs=5, batch_size=16)

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 651)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_4       │ (None, 651)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 651, 300)  │  4,500,000 │ input_layer_4[0]… │
│ (Embedding)         │                   │            │ input_layer_5[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_9         │ (None, 651, 300)  │          0 │ embedding[4][0]   │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ masking_4 (Masking) │ (None, 651, 300)  │          0 │ embedding[4][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ any_4 (Any)         │ (None, 651)       │          0 │ not_equal_9[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ masking_5 (Masking) │ (None, 651, 300)  │          0 │ embedding[5][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_4 (LSTM)       │ [(None, 512),     │  1,665,024 │ masking_4[0][0],  │
│                     │ (None, 512),      │            │ any_4[0][0]       │
│                     │ (None, 512)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_5 (LSTM)       │ [(None, 651,      │  1,665,024 │ masking_5[0][0],  │
│                     │ 512), (None,      │            │ lstm_4[0][1],     │
│                     │ 512), (None,      │            │ lstm_4[0][2]      │
│                     │ 512)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 651,       │  7,695,000 │ lstm_5[0][0]      │
│                     │ 15000)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 15,525,048 (59.22 MB)

 Trainable params: 11,025,048 (42.06 MB)

 Non-trainable params: 4,500,000 (17.17 MB)

Epoch 1/5
529/529 ━━━━━━━━━━━━━━━━━━━━ 86s 161ms/step - loss: 5.7282 - masked_accuracy: 0.2186 - val_loss: 4.8127 - val_masked_accuracy: 0.3116
Epoch 2/5
529/529 ━━━━━━━━━━━━━━━━━━━━ 85s 161ms/step - loss: 4.4400 - masked_accuracy: 0.3153 - val_loss: 4.8183 - val_masked_accuracy: 0.3200
Epoch 3/5
529/529 ━━━━━━━━━━━━━━━━━━━━ 86s 163ms/step - loss: 4.2001 - masked_accuracy: 0.3239 - val_loss: 4.7906 - val_masked_accuracy: 0.3260
Epoch 4/5
529/529 ━━━━━━━━━━━━━━━━━━━━ 104s 197ms/step - loss: 3.9787 - masked_accuracy: 0.3306 - val_loss: 4.7950 - val_masked_accuracy: 0.3283
Epoch 5/5
529/529 ━━━━━━━━━━━━━━━━━━━━ 134s 254ms/step - loss: 3.8127 - masked_accuracy: 0.3283 - val_loss: 4.7927 - val_masked_accuracy: 0.3313


In [41]:
model.evaluate([encoder_input_test, decoder_input_test], decoder_output_test, batch_size=16)

67/67 ━━━━━━━━━━━━━━━━━━━━ 6s 83ms/step - loss: 4.6726 - masked_accuracy: 0.3316


[4.717677593231201, 0.3264451026916504]

In [42]:
model.save("seq2seq_model.keras")

In [39]:
custom_objects = {
    'masked_accuracy':masked_accuracy
}

In [40]:
model = tf.keras.models.load_model("seq2seq_model.keras", custom_objects=custom_objects)

In [68]:
def indices_to_text(predictions, text_vectorizer):
    predictions = predictions.flatten()
    vocab = text_vectorizer.get_vocabulary()
    
    words = []
    for idx in predictions:
        if idx == 0:
            continue
        if idx >= len(vocab):
            continue
        
        word = vocab[idx]

        if word in ['<start>', '<end>']:
            continue
            
        words.append(word)
    
    return ' '.join(words)

In [48]:
predictions = model.predict([encoder_input_test[1:5], decoder_input_test[1:5]])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step


In [49]:
predicted_indices = np.argmax(predictions, axis=-1)

In [46]:
predicted_indices[2]

array([    1,    15,     0,    15,    15,    15,    15,    15,    15,
          15,    15,    15,    15,    15,    15,    15,    15,    15,
          15,    15,    15,    15,    15,    15,    15,    15,    15,
          15,    15,    15,    15,    15,    15,    15,    15,    15,
          15,    15,    15,    15,    15,    15,    15,    15,    15,
          15,    15,    15,    15,    15,    15,    15,    15,    15,
          15,    15,    15,    15,    15,    15,    15,    15, 12063,
       12063,  2271,  2271,  2271,  2271,  2271,     1,     1,    15,
          15,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0, 12073, 12063,  4203,  7323,  2271,     1,    15,
          15,    15,     0,     0,     0,     0,     0,     0,     0,
           0, 12063, 12063,  4203,  2271,  2271,     1,    15,    15,
          15,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0, 12063,  4203,  7323,  2271,     1,    15,    15,
          15,     0,

In [63]:
vocabulary[12063], embedding_matrix[12063][:3], embedding_matrix[1][:3]

('9317',
 array([ 0.01292978, -0.0196126 ,  0.03690822]),
 array([ 0.01292978, -0.0196126 ,  0.03690822]))

In [69]:
indices_to_text(predicted_indices, text_vectorizer)

'[UNK] fee zone zone zone zone zone zone [UNK] 4739N 9317 custom conscientious zone [UNK] 9317 custom zone zone [UNK] 9317 9317 custom zone zone [UNK] 9317 Chamberlain custom zone zone [UNK] 9317 custom conscientious zone [UNK] 9317 custom zone zone [UNK] 9317 IP conscientious zone zone [UNK] 9317 9317 custom zone zone [UNK] 9317 IP conscientious zone zone [UNK] 9317 custom zone zone [UNK] 9317 IP custom zone zone [UNK] 9317 IP conscientious zone [UNK] [UNK] 9317 custom zone zone [UNK] 9317 IP conscientious zone [UNK] [UNK] 9317 IP conscientious zone zone [UNK] 9317 IP conscientious zone zone [UNK] 9317 IP conscientious zone zone [UNK] 9317 IP conscientious zone zone [UNK] 9317 IP conscientious zone zone [UNK] 9317 IP conscientious zone zone [UNK] 9317 9317 custom zone zone [UNK] 9317 IP custom zone zone [UNK] 9317 custom conscientious zone [UNK] 9317 9317 custom zone zone [UNK] 9317 9317 custom zone zone [UNK] 9317 custom zone zone [UNK] 9317 9317 custom zone zone [UNK] 9317 9317 cust

In [107]:
embedding_layer = tf.keras.layers.Embedding(
    input_dim=vocab_size,
    output_dim=embedding_dim,
    weights=[embedding_matrix],
    input_length=max_len,
    mask_zero=False,
    trainable=False
)

In [62]:
# Encoder
encoder_input = tf.keras.layers.Input(shape=(max_len,))
encoder_embedding = embedding_layer(encoder_input)
_, forward_h, forward_c = tf.keras.layers.LSTM(512, return_state=True)(encoder_embedding)
encoder_states = [forward_h, forward_c]

# Decoder
decoder_input = tf.keras.layers.Input(shape=(max_len,))
decoder_embedding = embedding_layer(decoder_input)
decoder_vectors, _, _ = tf.keras.layers.LSTM(512, return_sequences=True, return_state=True)(decoder_embedding, initial_state=encoder_states)
decoder_output = tf.keras.layers.Dense(vocab_size, activation="softmax")(decoder_vectors)

model1 = tf.keras.Model([encoder_input, decoder_input], decoder_output)

model1.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=[masked_accuracy])

model1.summary()

model1.fit([encoder_input_train, decoder_input_train], decoder_output_train, validation_data=([encoder_input_val, decoder_input_val], decoder_output_val), epochs=5, batch_size=16)

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7       │ (None, 651)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_6       │ (None, 651)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 651, 300)  │  4,500,000 │ input_layer_6[0]… │
│ (Embedding)         │                   │            │ input_layer_7[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_6 (LSTM)       │ [(None, 512),     │  1,665,024 │ embedding_1[0][0] │
│                     │ (None, 512),      │            │                   │
│                     │ (None, 512)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_7 (LSTM)       │ [(None, 651,      │  1,665,024 │ embedding_1[1][0… │
│                     │ 512), (None,      │            │ lstm_6[0][1],     │
│                     │ 512), (None,      │            │ lstm_6[0][2]      │
│                     │ 512)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 651,       │  7,695,000 │ lstm_7[0][0]      │
│                     │ 15000)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 15,525,048 (59.22 MB)

 Trainable params: 11,025,048 (42.06 MB)

 Non-trainable params: 4,500,000 (17.17 MB)

Epoch 1/5
529/529 ━━━━━━━━━━━━━━━━━━━━ 103s 191ms/step - loss: 0.7405 - masked_accuracy: 0.0937 - val_loss: 0.0382 - val_masked_accuracy: 0.2948
Epoch 2/5
529/529 ━━━━━━━━━━━━━━━━━━━━ 124s 235ms/step - loss: 0.0362 - masked_accuracy: 0.3023 - val_loss: 0.0368 - val_masked_accuracy: 0.3120
Epoch 3/5
529/529 ━━━━━━━━━━━━━━━━━━━━ 100s 189ms/step - loss: 0.0419 - masked_accuracy: 0.3122 - val_loss: 0.0375 - val_masked_accuracy: 0.3117
Epoch 4/5
529/529 ━━━━━━━━━━━━━━━━━━━━ 93s 176ms/step - loss: 0.0350 - masked_accuracy: 0.3096 - val_loss: 0.0360 - val_masked_accuracy: 0.3186
Epoch 5/5
529/529 ━━━━━━━━━━━━━━━━━━━━ 103s 194ms/step - loss: 0.0320 - masked_accuracy: 0.3225 - val_loss: 0.0359 - val_masked_accuracy: 0.3199
